In [ ]:
import pandas as pd
df_collab = pd.read_csv('C:\\Users\\Brend\\Downloads\\pct_collaborative_pubs_top42.csv', index_col = 'name', encoding = "UTF-8")
df_collab_new = pd.read_csv('C:\\Users\\Brend\\Downloads\\pct_collaborative_pubs_top42_v2.csv', index_col = 'name', encoding = "UTF-8")
df_disrupt = pd.read_csv('C:\\Users\\Brend\\Downloads\\disrupt_top42.csv', index_col = 'name', encoding = "UTF-8")
df_impact = pd.read_csv('C:\\Users\\Brend\\Downloads\\impact_top42.csv', index_col = 'name', encoding = "UTF-8")
df_clusters = pd.read_csv('C:\\Users\\Brend\\Downloads\\clusters_top42.csv', index_col = 'name', encoding = "UTF-8")

In [ ]:
all_dfs = [df_collab_new, df_disrupt, df_impact, df_clusters]
df = all_dfs[0]
for i in range(1, len(all_dfs)):
    df = df.join(all_dfs[i], how = 'outer')

In [ ]:
print("Total: %d authors" % df.shape[0])
df_trim = df[df['num_pubs'] >= 50]
df_trim = df_trim[df_trim['TotalConns'] >= 25]
print("Remaining after trimming: %d authors" % df_trim.shape[0])

In [ ]:
def top_100_to_csv(df, name, to_csv = False):
    global df_trim
    d = dict()
    for col in df.columns:
        df_notna = df_trim[pd.notna(df_trim[col])]
        sorted = df_notna[[col]].sort_values(col, ascending = False).head(100)
        names = sorted.index.tolist()
        values = sorted[col].tolist()
        d[col] = pd.Series(names, index=list(range(1, 101)))
        d[col+'_val'] = pd.Series(values, list(range(1, 101)))
    df_rank = pd.DataFrame(d)
    if to_csv:
        df_rank.to_csv('C:\\Users\\Brend\\Downloads\\{}_rank.csv'.format(name), index = False, encoding = "UTF-8")
    return df_rank

In [ ]:
top_100_to_csv(df_collab_new, 'collab', to_csv = True)

In [ ]:
top_100_to_csv(df_impact, 'impact', to_csv = True)

In [ ]:
top_100_to_csv(df_disrupt, 'disrupt', to_csv = True)

In [ ]:
interesting_authors = ['Eric S. Lander', 'Robert Langer', 'Francis Crick', 'Sydney Brenner', 'Richard Henderson', 'Edward S. Boyden']
def find_percentiles(df, authors = interesting_authors):
    global df_trim
    df_percentile = df_trim.copy()
    for col in df_trim.columns:
        percentile_col = df_trim[pd.notna(df_trim[col])][col]
        df_percentile[col+'_%'] = percentile_col.rank(pct=True)
    df_percentile_authors = df_percentile.loc[authors][[col+'_%' for col in df.columns]]
    return df_percentile_authors

In [ ]:
find_percentiles(df_collab_new)

In [ ]:
find_percentiles(df_impact)

In [ ]:
find_percentiles(df_disrupt)